In [ ]:
!pip install gradio pandas numpy langchain sentence-transformers scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 28.

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Constants for default values
DEFAULT_CHUNK_SIZE = 100
DEFAULT_CHUNK_OVERLAP = 0
DEFAULT_NUM_CHUNKS = 10

# nomic-ai/nomic-embed-text-v1',trust_remote_code=True

# Initialize the sentence transformer model for embeddings
#model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('BAAI/bge-small-en-v1.5')
#model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
#model = SentenceTransformer('nomic-ai/nomic-embed-text-v1',trust_remote_code=True)


def tokenize_text(method, text, chunk_size, chunk_overlap, num_chunks):
    """
    Tokenizes the input text based on the selected method and provided parameters.
    """
    num_chunks = int(num_chunks)
    output = []

    # Ensure text is provided
    if not text.strip():
        return pd.DataFrame(columns=['Chunk #', 'Text Chunk', 'Character Count', 'Token Count'])

    if method == "RecursiveCharacterTextSplitter":
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len, is_separator_regex=False)
        tokenized_texts = text_splitter.split_text(text)[:num_chunks]
        for i, chunk in enumerate(tokenized_texts):
            output.append({
                'Chunk #': i,
                'Text Chunk': chunk,
                'Character Count': len(chunk),
                'Token Count': len(chunk.split())
            })

    df = pd.DataFrame(output)
    return df

def calculate_embeddings(df):
    """
    Calculates embeddings for each text chunk in the dataframe.
    """
    if df.empty:
        return df

    chunks = df['Text Chunk'].tolist()
    embeddings = model.encode(chunks)
    df['Embeddings'] = embeddings.tolist()
    return df

def search_similar_chunks(query, df_with_embeddings):
    """
    Search for chunks similar to the query embedding.
    """
    # Compute the query embedding
    query_embedding = model.encode([query])[0]

    # Calculate similarity scores
    chunk_embeddings = np.vstack(df_with_embeddings['Embeddings'])
    similarity_scores = cosine_similarity([query_embedding], chunk_embeddings)[0]

    # Insert similarity scores into the dataframe after 'Chunk #'
    df_with_embeddings.insert(1, 'Similarity', similarity_scores)

    # Return the dataframe sorted by similarity scores in descending order
    return df_with_embeddings.sort_values(by='Similarity', ascending=False)

def process_and_embed(method, text, chunk_size, chunk_overlap, num_chunks):
    """
    Tokenizes the text and calculates embeddings.
    """
    df = tokenize_text(method, text, chunk_size, chunk_overlap, num_chunks)
    df_with_embeddings = calculate_embeddings(df)
    return df_with_embeddings

def update_output(method, text, chunk_size, chunk_overlap, num_chunks, query):
    df_with_embeddings = process_and_embed(method, text, chunk_size, chunk_overlap, num_chunks)
    if query:
        df_with_embeddings = search_similar_chunks(query, df_with_embeddings)
        # Update the headers to reflect the new column order after similarity search
        return df_with_embeddings[['Chunk #', 'Similarity', 'Text Chunk', 'Character Count', 'Token Count', 'Embeddings']]
    return df_with_embeddings[['Chunk #', 'Text Chunk', 'Character Count', 'Token Count', 'Embeddings']]

iface = gr.Interface(
    fn=update_output,
    inputs=[
        gr.Dropdown(label="Select Tokenization Method", choices=["RecursiveCharacterTextSplitter"]),
        #gr.Textbox(label="Enter Text", lines=10, placeholder="Type or paste text here."),
        gr.Textbox(label="Enter Text", value=DEFAULT_TEXT1),
        gr.Number(label="Chunk Size", value=DEFAULT_CHUNK_SIZE),
        gr.Number(label="Chunk Overlap", value=DEFAULT_CHUNK_OVERLAP),
        gr.Number(label="Number of Chunks to Display", value=DEFAULT_NUM_CHUNKS),
        gr.Textbox(label="Enter Query for Similarity Search", lines=2, placeholder="Type your query here.")
    ],
    outputs=gr.Dataframe(height=900),
    title="Text Tokenization and Embedding Tool",
    description="A tool for tokenizing text and calculating embeddings. Now with similarity search feature."
)

if __name__ == "__main__":
    iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7b4ebd92bac428ad36.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
DEFAULT_TEXT2 = """
Deming’s Journey to Profound Knowledge - How Deming Helped Win a War, Altered the Face of Industry, and Holds the Key to Our Future - Part 1 - Chapter 1: Humble Origins and Non-Determinism

Deming’s Humble Beginnings

Deming’s one childhood claim to fame was when Buffalo Bill recognized him in the crowd during a performance of “Buffalo Bill’s Wild West Show” outside Los Angeles where “Edwards,” as his family called him, was visiting his cousins.

The notoriously flamboyant showman knew the boy from Cody, Wyoming, by sight if not by name. Buffalo Bill was arguably the most world-famous living American at the time, having extensively toured the US and then Europe, performing before Queen Victoria herself twice. His act made him not only famous but rich. But Buffalo Bill wanted to be taken seriously as a legitimate businessman instead of a circus act. He used his money and influence in an enterprise to create the largest undertaking of its kind ever attempted in the West: using irrigation to create an agricultural empire stretching along the Shoshone River. He began by incorporating a small town in 1896 about fifty miles east of Yellowstone National Park, which Colonel William Frederick “Buffalo Bill” Cody humbly named after himself. Life in and around Cody, Wyoming, revolved around Bill, and Bill was usually in and around “the sweetest hotel that ever was.”1

Named after his daughter, the Irma Hotel not only housed travelers but served as Bill’s headquarters, comprising two personal suites and his professional offices. His hotel was, in effect, the heart of his endeavor. When he built it, he envisaged something akin to an African city serving as a staging point for safari expeditions, perhaps like Zanzibar or Mombasa. He foresaw hosting European nobility hunting big game, East Coast financiers looking for potential investments, and opportunists from everywhere investigating the mining and ranching possibilities. The bread and butter of the Irma, however, would be the promised flood of newcomers flocking to settle the soon-to-be-verdant plains around Cody.

Shortly after the Irma opened in 1902, the Demings arrived from Sioux City, Iowa. The thirty-three-year-old father, William, had been trained as a law clerk but now sought to make his fortune on the frontier. He arranged for temporary employment with an attorney in Cody, then moved his wife and two toddling boys from the breadbasket of America to the barren badlands of Wyoming. The town was still in its infancy. There wasn’t enough legal work to keep the young father employed full time, so he found a job at the Irma as a sort of jack-of-all-trades. In addition to his wages, the hotel provided him and his family with a small house on the grounds. Edwards and his brother became regular fixtures at the Irma. Thus, Buffalo Bill recognized Edwards and his little brother at the LA performance.

Unfortunately for Buffalo Bill, the Cody-based irrigation empire failed. But in 1905, the federal government began a massive public works project via the US Reclamation Service aimed at irrigating ninety thousand acres to turn the semi-arid Bighorn Basin into fertile farms. This necessitated the construction of the Shoshone River Dam twenty-five miles northeast of Cody, around the settlement of Powell. Once completed, the concrete-arch gravity dam—itself a predecessor to the Hoover Dam—was the tallest dam in the world.

The area around Powell was opened to homesteaders, and, in 1906, Mr. William Deming applied for and received forty acres of farmland on the edge of town. Or at least what everyone hoped would be farmland one day. In the meantime, the Demings eked out what living they could out where the Great Plains meet the Rocky Mountains.

Decades later, Ed recalled his family’s hardscrabble life. “Our house in Powell, roughly 1908 to 1912, was a tarpaper shack about the size of a freight car. . . . Electricity and indoor plumbing were out of the question. Snow blew in through the cracks in the door and in the windows.” He recollected owning a cat at the time that slept with him and his brother, keeping them warm at night.

The Shoshone River Dam (also known as Buffalo Bill Dam) was completed in 1910, but with or without irrigation, Deming’s father was never a successful farmer. He once remarked, “A farmer makes his money on the farm and spends it in town. An agriculturalist makes his money in town and spends it on the farm . . . . I’m an agriculturalist.” The eggs, milk, and vegetables that kept the family alive came from their chickens, cow, and garden. Despite their efforts, it wasn’t always enough. Late in his life, Dr. Deming would recall his childhood. “I remember my mother, tak-
ing my brother and me by the hand, prayed for food.”

Elizabeth, his younger sister and the first baby born in Powell, later noted, “We didn’t have much, but nobody had anything.” To make ends meet, William continued to do some legal work in the area while Mrs. Pluma Deming, neé Edwards, taught piano and voice lessons on her Steinway parlor grand piano. William later began traveling, selling real estate and insurance. Over time, his business grew enough that the Demings were able to move out of the little tar shack on the prairie and into a slightly better home.

Though poor, his parents were well educated and poured their knowledge (and, perhaps, thirst for more) into their children. Edwards was raised in an atmosphere that included both left-brain and right-brain learning—his mother provided the right-brain perspective— the synthetic and creative aspects of learning . . . through music, while his father tended more toward the left-brain (cognitive) perspective. The atmosphere created by Deming’s parents served as the basis for his intellectual achievements and quite likely spurred the qualities which contributed to his success—an intense work ethic, devotion to spouse and family, a love of music. This environment set the course for Deming’s life. His hard-knocks upbringing gave him a unique perspective on the on-the-ground reality of the working class that managers—especially those from more privileged backgrounds—couldn’t appreciate. And the atmosphere his parents fostered helped him became a philomath: a lifelong learner and boundary spanner, a master of engineering and statistics in addition to a musician, composer, and linguist. This all resulted in a young man who was serious, studious, and diligent. The family even came to (prophetically) nickname him “the Professor.”

Newton’s Apple and Schrödinger’s Cat Set the Stage The year before the Demings moved from the Irma to the tar shack in Powell, a theoretical physicist published four groundbreaking papers that challenged the laws of physics (and would be instrumental in framing much of the thinking in Deming’s Profound Knowledge). One of the papers held the seeds of what would become the world’s most famous formula, E=mc2.

Before Einstein, everyone relied on Sir Isaac Newton’s explanations of how the physical world works (e.g., for every action there’s an equal and opposite reaction, objects in motion tend to stay in motion unless acted upon by an outside force, etc.). Einstein showed the world that physics is more like the Korean DMZ: Newton’s laws applied only in certain jurisdictions. Past that border was a whole other world. That border was the atom, the basic building block of all matter. Newtonian physics governed the apple: If you drop it, it will fall to the ground. However, once you get to the quantum level, everything goes squirrely. You can’t be certain what subatomic particles will do. For example, from the subatomic perspective, if you drop an apple, it may or may not hit the ground. It’s enough to make your head hurt.

Others quickly built on Einstein’s work throughout the 1920s, including Niels Bohr (who later worked on the Manhattan Project) and Erwin Schrödinger. But these physicists’ discoveries were just pieces of a much bigger shift: the rise of non-determinism.

Before Einstein published his famous E=mc2, and his following Nobel prize for his discovery of the law of the photoelectric effect, the world was seen through the lens of determinism (Newtonian Physics): “If I drop this apple, it will fall.” In simpler terms, the world operates solely on cause and effect. Take the weather, for instance. Decades ago, meteorologists believed that if they knew all the variables, such as humidity, wind direction, barometric pressure, etc., they could predict the weather with 100% accuracy. In fact, the earliest computers were created expressly for calculating all these variables. But even with the advances in technology we have today, meteorologists are never spot-on all the time. Even if we can calculate every variable, there’s still randomness, non-determinism.

Non-determinism also has roots in Charles Darwin’s Theory of Evolution: If you cross a black cow with a black cow, the offspring will probably be another black cow . . . but a gene might mutate and result in a two-headed white calf. You just can’t ever be certain. This is what physicist Max Planck (the father of quantum mechanics), Einstein, and others observed: No matter how much you know, there is an infinite amount of chance and randomness in the universe. Therefore, there can be no such thing as absolute certainty; the world is constantly in flux. This academic environment prepared Deming to pursue probability and statistics, a cornerstone to his Theory of Variation.

Niels Bohr and Werner Heisenberg took this idea of infinite variability to its extreme with the Copenhagen interpretation, which states that a quantum particle does not exist in one state or another, but in all of its possible states at the same time.

Erwin Schrödinger gave us an easy way to understand how those two physicists saw the way the world works (the difference between deterministic thinking and non-deterministic thinking). Say you put a cat in a sealed box. Inside, there are two items. One is a can of poisonous gas. The other is a radioactive isotope giving off gamma rays. When the isotope decays and releases gamma rays, it triggers the poison gas. The cat dies. The catch is you can’t predict when the isotope will decay.

Radioactive decay can be random; no two isotopes decay at the same rate. It could decay in a minute or in a thousand years. Therefore, you’ll never know when the isotope in the box will decay, triggering the poisonous gas. According to Bohr and Heisenberg, since you can’t predict when the element will decay, you can never be sure at any given moment whether the cat is dead or alive. Until you open the box to see for yourself, you have to simultaneously assume that the cat is alive and that it’s dead. Schrödinger’s thought experiment here was to show the absurdity of those physicists’ extreme and extremely theoretical view.

While this cat-in-the-box concept is funny, it illustrates how these two schools of thought differed. Determinism saw the world in black and white, cause and effect. With enough information, you could control any situation.

Non-determinism sees the world in shades of gray. Everything has an element of randomness. Much of how the world works is unknowable. Mathematical formulas don’t always hold true; we can’t accurately predict the future. We can only speak in probabilities: “The apple will more than likely hit the ground, but we can’t say that with 100% certainty.”

This idea of non-determinism—that reality is inherently random—would form the basis of Deming’s worldview when he began his academic career. It taught him to see the world as a series of interconnected systems, sparking the beginning of his questioning knowledge and leading to the first element of the System of Profound Knowledge: How do we know what we know?

Missing the Forest for the Trees

Let’s look at a real-life example of non-determinism.

Post–World War II, the island of Borneo in Southeast Asia had a serious malaria problem. In 1952, the World Health Organization (WHO) of the newly formed United Nations sent antimalarial experts to address the situation. One of the primary carriers of malaria is mosquitoes. Over the next three years, the WHO sprayed the chemical pesticide DDT on interior surfaces in the village longhouses, each of which housed about a hundred families. After malaria cases sharply declined, the WHO declared the mission accomplished and proceeded to host a world assembly in Mexico City to extol the virtues of DDT.

Five years after the conference, Borneo started raining cats. Literally.

And not just any cats. These were special cats: twenty-three rat catchers that floated down in their very own little cat parachutes from a British Royal Air Force transport plane.

The cats’ mission: to replenish the island’s feline population. What happened to the native cats? As it turns out, DDT had killed more than mosquitoes. Later autopsies revealed that the WHO’s practices resulted in lethal amounts of DDT accumulating in cats. Without their natural predator, the rat population exploded. Rats don’t just eat crops; they carry diseases. In Borneo’s case, typhus and sylvatic plague (the same bacterium that caused the bubonic plague of Black Plague fame). Nature could reset the ratio of cats to rats elsewhere, perhaps, but Borneo is an island. If all the cats die, there are no more cats. To remedy the WHO’s mistake, the RAF flew twenty-three cats (plus three tons of food and supplies), blessed them to “go forth and multiply,” and let ’er rip.

I imagine it was a carnival of carnivorous and carnal delight. The good folks at the WHO made an honest mistake. After all, the Western education system stresses analytical, deterministic thinking. In this case, it led to this line of reasoning:

1. Malaria is bad in Borneo.
2. Malaria is carried by mosquitoes.
3. DDT kills mosquitoes.
4. Therefore, we should use DDT to kill the mosquitoes in Borneo.

Cats killing rats and therefore keeping typhus at manageable levels is what Donella Meadows, in Thinking in Systems, calls a balancing feedback loop. However, when the cat population was out of balance, the natural order of things oscillated, creating what she describes as an overshoot of a reinforcing feed-back loop.

If the WHO had embraced non-deterministic thinking, they would have taken a much wider view of the problem. The opposite of analytic thinking is systems thinking (a.k.a. appreciation of a system): the ability to see how one thing is part of a larger, connected system. Someone who approached the ecosystem as a system might have thought along these lines:

1. Malaria is bad in Borneo.
2. Malaria is carried by mosquitoes.
3. DDT kills mosquitoes . . . but what else could it kill?
4. What else would spraying DDT on the inside of longhouses affect?
5. Do we have enough information to make an overall decision?
6. We should hold back until we can be reasonably sure we’re going to make things better and not worse for the people of Borneo.

The WHO focused only on the immediate problem and failed to consider how one “solution” might trigger a chain reaction. They failed to see the whole system. This is exactly what I meant earlier about Profound Knowledge: profound change requires Profound Knowledge, and one of the tenets of Profound Knowledge is systems thinking, an ability to see the situation in its greater context.

Determinism and analytical thinking break down a problem into tiny pieces, whereas non-determinism and systems thinking look at a problem’s bigger picture.

Analytical thinkers say, “Mission accomplished. Now, let’s go home.”

Systems thinkers say, “What were the results? Now, let’s make it even
better.”

This was bleeding-edge thinking when a sixteen-year-old’s train rolled into Laramie, Wyoming. “The Professor” was going to college.

The Professor Becomes the Student

Ed, as he would come to introduce himself, was used to shouldering a heavy load. He expected things to be no different at the University of Wyoming. In fact, he decided to major in electrical engineering. Electricity at the time was still at the forefront of technological progress, so this was like majoring in artificial intelligence or quantum computing today.

As he studied electrical engineering over the next four years, he supported himself by working as a janitor, shoveling snow, and cutting ice. He also cut railroad ties and worked at a dry cleaner. At some point, he was a soda jerk serving up malted milkshakes. On top of working and studying, he also sang in a church choir and played the piccolo in the university’s marching band. This blue-collar work ethic as well as his continued pursuit of service and the arts were a recurring pattern throughout his life. Ed was self-sufficient yet always found time to help those in need.

Ed graduated in four years but stayed for a fifth to study mathematics before enrolling at the University of Colorado for a master’s in physics and mathematics. After he graduated with his master’s in 1924, one of his instructors encouraged him to continue with his studies, perhaps at Yale. He moved to New Haven, where, three years later, he would earn his PhD in mathematical physics, the basis of probabilities and statistics and the backbone of non-deterministic thinking.

The 1920s were an exciting time for scientific discovery. The year Ed received his PhD, the fifth Solvay Conference on Physics was held. The subject was electrons and photons. In attendance were some of the most famous names in science to this day, including Marie Curie, Edwin Schrödinger, Max Planck, Albert Einstein, Niels Bohr, and Werner Heisenberg. The conference spawned an explosion in scientific thought and discovery based on non-determinism.

Non-determinism played a crucial role in shaping Deming’s worldview and began to lay the foundations for his System of Profound Knowledge. For one, it taught him that long-established and long-held beliefs weren’t necessarily true; the entire structure of the physical world was being rethought and reexamined.

Second, it showed him that the underpinnings of our very existence are random. That idea of randomness would be born out through his fascination with statistics, which in turn would inform his understanding of variation (the second element in the System of Profound Knowledge).

Third, it taught him to look beyond black-and-white cause and effect. It forced him to look at problems as multifaceted, complex systems, where changing one factor might have far-reaching, and unintended, consequences. This was the beginnings of his understanding of the fourth element of Profound Knowledge: Systems Thinking.

But before we go forward, we need to rewind briefly. During the two summers bracketing Yale’s academic calendar, Ed Deming—a university faculty member with a bachelor’s in engineering, a master’s in physics and was working on a PhD—supported himself and his wife (he’d married a schoolteacher, Agnes, in 1923), as ever, by working.

Then, Ed took an internship in a Chicago sweatshop: Hawthorne Works.
"""

DEFAULT_TEXT1 = """
The Godfather
1. The Godfather (1972)
R | 175 min | Crime, Drama

 9.2  Rate 100 Metascore
The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.

Director: Francis Ford Coppola | Stars: Marlon Brando, Al Pacino, James Caan, Diane Keaton

Votes: 1,995,042 | Gross: $134.97M

 Watch on Paramount+
with Prime Video Channels
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 3
Oscar Nominations: 11
BAFTA Awards: 0
BAFTA Nominations: 4
Golden Globes: 6
Golden Globe Nominations: 8

The Shawshank Redemption
2. The Shawshank Redemption (1994)
R | 142 min | Drama

 9.3  Rate 82 Metascore
Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.

Director: Frank Darabont | Stars: Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler

Votes: 2,864,095 | Gross: $28.34M

Actors: 4.8 Stars
Direction: 5 Stars
Screenplay: 4.9 Stars

Oscars: 0
Oscar Nominations: 7
BAFTA Awards: 0
BAFTA Nominations: 0
Golden Globes: 0
Golden Globe Nominations: 2

Schindler's List
3. Schindler's List (1993)
R | 195 min | Biography, Drama, History

 9  Rate 95 Metascore
In German-occupied Poland during World War II, industrialist Oskar Schindler gradually becomes concerned for his Jewish workforce after witnessing their persecution by the Nazis.

Director: Steven Spielberg | Stars: Liam Neeson, Ralph Fiennes, Ben Kingsley, Caroline Goodall

Votes: 1,438,633 | Gross: $96.90M

Actors: 4.9 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 7
Oscar Nominations: 12
BAFTA Awards: 6
BAFTA Nominations: 12
Golden Globes: 3
Golden Globe Nominations: 6

Raging Bull
4. Raging Bull (1980)
R | 129 min | Biography, Drama, Sport

 8.1  Rate 90 Metascore
The life of boxer Jake LaMotta, whose violence and temper that led him to the top in the ring destroyed his life outside of it.

Director: Martin Scorsese | Stars: Robert De Niro, Cathy Moriarty, Joe Pesci, Frank Vincent

Votes: 377,546 | Gross: $23.38M

Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 2
Oscar Nominations: 8
BAFTA Awards: 2
BAFTA Nominations: 4
Golden Globes: 1
Golden Globe Nominations: 7

Casablanca
5. Casablanca (1942)
PG | 102 min | Drama, Romance, War

 8.5  Rate 100 Metascore
A cynical expatriate American cafe owner struggles to decide whether or not to help his former lover and her fugitive husband escape the Nazis in French Morocco.

Director: Michael Curtiz | Stars: Humphrey Bogart, Ingrid Bergman, Paul Henreid, Claude Rains

Votes: 602,326 | Gross: $1.02M

Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 3
Oscar Nominations: 8
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

Citizen Kane
6. Citizen Kane (1941)
PG | 119 min | Drama, Mystery

 8.3  Rate 100 Metascore
Following the death of publishing tycoon Charles Foster Kane, reporters scramble to uncover the meaning of his final utterance: 'Rosebud.'

Director: Orson Welles | Stars: Orson Welles, Joseph Cotten, Dorothy Comingore, Agnes Moorehead

Votes: 463,322 | Gross: $1.59M

Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 1
Oscar Nominations: 9
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

Gone with the Wind
7. Gone with the Wind (1939)
Passed | 238 min | Drama, Romance, War

 8.2  Rate 97 Metascore
A sheltered and manipulative Southern belle and a roguish profiteer face off in a turbulent romance as the society around them crumbles with the end of slavery and is rebuilt during the Civil War and Reconstruction periods.

Directors: Victor Fleming, George Cukor, Sam Wood | Stars: Clark Gable, Vivien Leigh, Thomas Mitchell, Barbara O'Neil

Votes: 332,169 | Gross: $198.68M

*****
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 8
Oscar Nominations: 13
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

The Wizard of Oz
8. The Wizard of Oz (1939)
PG | 102 min | Adventure, Family, Fantasy

 8.1  Rate 92 Metascore
Young Dorothy Gale and her dog Toto are swept away by a tornado from their Kansas farm to the magical Land of Oz, and embark on a quest with three new friends to see the Wizard, who can return her to her home and fulfill the others' wishes.

Directors: Victor Fleming, King Vidor | Stars: Judy Garland, Frank Morgan, Ray Bolger, Bert Lahr

Votes: 425,800 | Gross: $2.08M

*****
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 2
Oscar Nominations: 6
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

One Flew Over the Cuckoo's Nest
9. One Flew Over the Cuckoo's Nest (1975)
R | 133 min | Drama

 8.7  Rate 84 Metascore
In the Fall of 1963, a Korean War veteran and criminal pleads insanity and is admitted to a mental institution, where he rallies up the scared patients against the tyrannical nurse.

Director: Milos Forman | Stars: Jack Nicholson, Louise Fletcher, Michael Berryman, Peter Brocco

Votes: 1,065,285 | Gross: $112.00M

*****
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 5
Oscar Nominations: 9
BAFTA Awards: 6
BAFTA Nominations: 9
Golden Globes: 6
Golden Globe Nominations: 6

Lawrence of Arabia
10. Lawrence of Arabia (1962)
Approved | 218 min | Adventure, Biography, Drama

 8.3  Rate 100 Metascore
The story of T.E. Lawrence, the English officer who successfully united and led the diverse, often warring, Arab tribes during World War I in order to fight the Turks.

Director: David Lean | Stars: Peter O'Toole, Alec Guinness, Anthony Quinn, Jack Hawkins

Votes: 311,874 | Gross: $44.82M

Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 7
Oscar Nominations: 10
BAFTA Awards: 4
BAFTA Nominations: 5
Golden Globes: 4
Golden Globe Nominations: 7

Vertigo
11. Vertigo (1958)
PG | 128 min | Mystery, Romance, Thriller

 8.3  Rate 100 Metascore
A former San Francisco police detective juggles wrestling with his personal demons and becoming obsessed with the hauntingly beautiful woman he has been hired to trail, who may be deeply disturbed.

Director: Alfred Hitchcock | Stars: James Stewart, Kim Novak, Barbara Bel Geddes, Tom Helmore

Votes: 424,261 | Gross: $3.20M

 Watch on Prime Video
rent/buy from $3.99
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 0
Oscar Nominations: 2
BAFTA Awards: 0
BAFTA Nominations: 0
Golden Globes: 0
Golden Globe Nominations: 0

Psycho
12. Psycho (1960)
R | 109 min | Horror, Mystery, Thriller

 8.5  Rate 97 Metascore
A Phoenix secretary embezzles $40,000 from her employer's client, goes on the run and checks into a remote motel run by a young man under the domination of his mother.

Director: Alfred Hitchcock | Stars: Anthony Perkins, Janet Leigh, Vera Miles, John Gavin

Votes: 713,471 | Gross: $32.00M

 Watch on Prime Video
rent/buy from $4.29
*****
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 0
Oscar Nominations: 4
BAFTA Awards: 0
BAFTA Nominations: 0
Golden Globes: 1
Golden Globe Nominations: 1

The Godfather Part II
13. The Godfather Part II (1974)
R | 202 min | Crime, Drama

 9  Rate 90 Metascore
The early life and career of Vito Corleone in 1920s New York City is portrayed, while his son, Michael, expands and tightens his grip on the family crime syndicate.

Director: Francis Ford Coppola | Stars: Al Pacino, Robert De Niro, Robert Duvall, Diane Keaton

Votes: 1,353,632 | Gross: $57.30M

Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 6
Oscar Nominations: 11
BAFTA Awards: 1
BAFTA Nominations: 3
Golden Globes: 0
Golden Globe Nominations: 6

On the Waterfront
14. On the Waterfront (1954)
Approved | 108 min | Crime, Drama, Thriller

 8.1  Rate 91 Metascore
An ex-prize fighter turned New Jersey longshoreman struggles to stand up to his corrupt union bosses, including his older brother, as he starts to connect with the grieving sister of one of the syndicate's victims.

Director: Elia Kazan | Stars: Marlon Brando, Karl Malden, Lee J. Cobb, Rod Steiger

Votes: 163,648 | Gross: $9.60M

*****
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 8
Oscar Nominations: 12
BAFTA Awards: 1
BAFTA Nominations: 3
Golden Globes: 4
Golden Globe Nominations: 4

Sunset Blvd.
15. Sunset Blvd. (1950)
Passed | 110 min | Drama, Film-Noir

 8.4  Rate 94 Metascore
A screenwriter develops a dangerous relationship with a faded film star determined to make a triumphant return.

Director: Billy Wilder | Stars: William Holden, Gloria Swanson, Erich von Stroheim, Nancy Olson

Votes: 234,860

*****
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 3
Oscar Nominations: 11
BAFTA Awards: 0
BAFTA Nominations: 0
Golden Globes: 4
Golden Globe Nominations: 7

Forrest Gump
16. Forrest Gump (1994)
PG-13 | 142 min | Drama, Romance

 8.8  Rate 82 Metascore
The history of the United States from the 1950s to the '70s unfolds from the perspective of an Alabama man with an IQ of 75, who yearns to be reunited with his childhood sweetheart.

Director: Robert Zemeckis | Stars: Tom Hanks, Robin Wright, Gary Sinise, Sally Field

Votes: 2,235,926 | Gross: $330.25M

Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 6
Oscar Nominations: 13
BAFTA Awards: 1
BAFTA Nominations: 7
Golden Globes: 3
Golden Globe Nominations: 7

The Sound of Music
17. The Sound of Music (1965)
G | 172 min | Biography, Drama, Family

 8.1  Rate 63 Metascore
A young novice is sent by her convent in 1930s Austria to become a governess to the seven children of a widowed naval officer.

Director: Robert Wise | Stars: Julie Andrews, Christopher Plummer, Eleanor Parker, Richard Haydn

Votes: 258,472 | Gross: $163.21M

*****
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 5
Oscar Nominations: 10
BAFTA Awards: 0
BAFTA Nominations: 1
Golden Globes: 2
Golden Globe Nominations: 4

12 Angry Men
18. 12 Angry Men (1957)
Approved | 96 min | Crime, Drama

 9  Rate 97 Metascore
The jury in a New York City murder trial is frustrated by a single member whose skeptical caution forces them to more carefully consider the evidence before jumping to a hasty verdict.

Director: Sidney Lumet | Stars: Henry Fonda, Lee J. Cobb, Martin Balsam, John Fiedler

Votes: 855,963 | Gross: $4.36M

*****
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 0
Oscar Nominations: 3
BAFTA Awards: 1
BAFTA Nominations: 2
Golden Globes: 0
Golden Globe Nominations: 4

West Side Story
19. West Side Story (1961)
Approved | 153 min | Crime, Drama, Musical

 7.6  Rate 86 Metascore
Two youngsters from rival New York City gangs fall in love, but tensions between their respective friends build toward tragedy.

Directors: Jerome Robbins, Robert Wise | Stars: Natalie Wood, George Chakiris, Richard Beymer, Russ Tamblyn

Votes: 120,507 | Gross: $43.66M

*****
Actors: 4.9 Stars
Direction: 4.9 Stars
Screenplay: 4.8 Stars

Oscars: 10
Oscar Nominations: 11
BAFTA Awards: 0
BAFTA Nominations: 1
Golden Globes: 3
Golden Globe Nominations: 5

Star Wars: Episode IV - A New Hope
20. Star Wars: Episode IV - A New Hope (1977)
PG | 121 min | Action, Adventure, Fantasy

 8.6  Rate 90 Metascore
Luke Skywalker joins forces with a Jedi Knight, a cocky pilot, a Wookiee and two droids to save the galaxy from the Empire's world-destroying battle station, while also attempting to rescue Princess Leia from the mysterious Darth Vader.

Director: George Lucas | Stars: Mark Hamill, Harrison Ford, Carrie Fisher, Alec Guinness

Votes: 1,439,775 | Gross: $322.74M

Actors: 4.8 Stars
Direction: 5 Stars
Screenplay: 4.9 Stars

Oscars: 6
Oscar Nominations: 10
BAFTA Awards: 1
BAFTA Nominations: 5
Golden Globes: 1
Golden Globe Nominations: 4

2001: A Space Odyssey
21. 2001: A Space Odyssey (1968)
G | 149 min | Adventure, Sci-Fi

 8.3  Rate 84 Metascore
After uncovering a mysterious artifact buried beneath the Lunar surface, a spacecraft is sent to Jupiter to find its origins: a spacecraft manned by two men and the supercomputer HAL 9000.

Director: Stanley Kubrick | Stars: Keir Dullea, Gary Lockwood, William Sylvester, Daniel Richter

Votes: 713,873 | Gross: $56.95M

Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 1
Oscar Nominations: 4
BAFTA Awards: 3
BAFTA Nominations: 5
Golden Globes: 0
Golden Globe Nominations: 0

E.T. the Extra-Terrestrial
22. E.T. the Extra-Terrestrial (1982)
PG | 115 min | Adventure, Family, Sci-Fi

 7.9  Rate 92 Metascore
A troubled child summons the courage to help a friendly alien escape from Earth and return to his home planet.

Director: Steven Spielberg | Stars: Henry Thomas, Drew Barrymore, Peter Coyote, Dee Wallace

Votes: 434,712 | Gross: $435.11M

*****
Actors: 4.8 Stars
Direction: 4.9 Stars
Screenplay: 4.9 Stars

Oscars: 4
Oscar Nominations: 9
BAFTA Awards: 1
BAFTA Nominations: 12
Golden Globes: 2
Golden Globe Nominations: 5

The Silence of the Lambs
23. The Silence of the Lambs (1991)
R | 118 min | Crime, Drama, Thriller

 8.6  Rate 86 Metascore
A young F.B.I. cadet must receive the help of an incarcerated and manipulative cannibal killer to help catch another serial killer, a madman who skins his victims.

Director: Jonathan Demme | Stars: Jodie Foster, Anthony Hopkins, Scott Glenn, Ted Levine

Votes: 1,535,177 | Gross: $130.74M

Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 5
Oscar Nominations: 7
BAFTA Awards: 2
BAFTA Nominations: 9
Golden Globes: 1
Golden Globe Nominations: 5

Chinatown
24. Chinatown (1974)
R | 130 min | Drama, Mystery, Thriller

 8.1  Rate 92 Metascore
A private detective hired to expose an adulterer in 1930s Los Angeles finds himself caught up in a web of deceit, corruption, and murder.

Director: Roman Polanski | Stars: Jack Nicholson, Faye Dunaway, John Huston, Perry Lopez

Votes: 347,175

 Watch on Paramount+
with Prime Video Channels
*****
Actors: 4.8 Stars
Direction: 4.8 Stars
Screenplay: 4.8 Stars

Oscars: 1
Oscar Nominations: 11
BAFTA Awards: 3
BAFTA Nominations: 10
Golden Globes: 4
Golden Globe Nominations: 7

The Bridge on the River Kwai
25. The Bridge on the River Kwai (1957)
PG | 161 min | Adventure, Drama, War

 8.1  Rate 87 Metascore
British POWs are forced to build a railway bridge across the river Kwai for their Japanese captors in occupied Burma, not knowing that the allied forces are planning a daring commando raid through the jungle to destroy it.

Director: David Lean | Stars: William Holden, Alec Guinness, Jack Hawkins, Sessue Hayakawa

Votes: 232,083 | Gross: $44.91M

 Watch on Prime Video
rent/buy from $3.59
Actors: 4.8 Stars
Direction: 4.9 Stars
Screenplay: 4.9 Stars

Oscars: 7
Oscar Nominations: 8
BAFTA Awards: 4
BAFTA Nominations: 4
Golden Globes: 3
Golden Globe Nominations: 4

Singin' in the Rain
26. Singin' in the Rain (1952)
G | 103 min | Comedy, Musical, Romance

 8.3  Rate 99 Metascore
A silent film star falls for a chorus girl just as he and his delusionally jealous screen partner are trying to make the difficult transition to talking pictures in 1920s Hollywood.

Directors: Stanley Donen, Gene Kelly | Stars: Gene Kelly, Donald O'Connor, Debbie Reynolds, Jean Hagen

Votes: 258,775 | Gross: $8.82M

 Watch on Max
with Prime Video Channels
*****
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 0
Oscar Nominations: 2
BAFTA Awards: 0
BAFTA Nominations: 1
Golden Globes: 1
Golden Globe Nominations: 2

It's a Wonderful Life
27. It's a Wonderful Life (1946)
PG | 130 min | Drama, Family, Fantasy

 8.6  Rate 89 Metascore
An angel is sent from Heaven to help a desperately frustrated businessman by showing him what life would have been like if he had never existed.

Director: Frank Capra | Stars: James Stewart, Donna Reed, Lionel Barrymore, Thomas Mitchell

Votes: 496,186

 Watch on Freevee
Watch Free on Freevee
*****
Actors: 5 Stars
Direction: 4.9 Stars
Screenplay: 4.9 Stars

Oscars: 0
Oscar Nominations: 5
BAFTA Awards: 0
BAFTA Nominations: 0
Golden Globes: 1
Golden Globe Nominations: 1

Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb
28. Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
PG | 95 min | Comedy, War

 8.4  Rate 97 Metascore
An insane American general orders a bombing attack on the Soviet Union, triggering a path to nuclear holocaust that a war room full of politicians and generals frantically tries to stop.

Director: Stanley Kubrick | Stars: Peter Sellers, George C. Scott, Sterling Hayden, Keenan Wynn

Votes: 514,985 | Gross: $0.28M

 Watch on Max
with Prime Video Channels
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 0
Oscar Nominations: 4
BAFTA Awards: 3
BAFTA Nominations: 6
Golden Globes: 0
Golden Globe Nominations: 0

Some Like It Hot
29. Some Like It Hot (1959)
Passed | 121 min | Comedy, Music, Romance

 8.2  Rate 98 Metascore
After two male musicians witness a mob hit, they flee the state in an all-female band disguised as women, but further complications set in.

Director: Billy Wilder | Stars: Marilyn Monroe, Tony Curtis, Jack Lemmon, George Raft

Votes: 281,859 | Gross: $25.00M

 Watch on Max
with Prime Video Channels
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 1
Oscar Nominations: 6
BAFTA Awards: 1
BAFTA Nominations: 2
Golden Globes: 3
Golden Globe Nominations: 3

Ben-Hur
30. Ben-Hur (1959)
G | 212 min | Adventure, Drama

 8.1  Rate 90 Metascore
After a Jewish prince is betrayed and sent into slavery by a Roman friend in 1st-century Jerusalem, he regains his freedom and comes back for revenge.

Director: William Wyler | Stars: Charlton Heston, Jack Hawkins, Stephen Boyd, Haya Harareet

Votes: 251,772 | Gross: $74.70M

 Watch on Prime Video
rent/buy from $3.99
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 11
Oscar Nominations: 12
BAFTA Awards: 1
BAFTA Nominations: 1
Golden Globes: 3
Golden Globe Nominations: 4

Apocalypse Now
31. Apocalypse Now (1979)
R | 147 min | Drama, Mystery, War

 8.4  Rate 94 Metascore
A U.S. Army officer serving in Vietnam is tasked with assassinating a renegade Special Forces Colonel who sees himself as a god.

Director: Francis Ford Coppola | Stars: Martin Sheen, Marlon Brando, Robert Duvall, Frederic Forrest

Votes: 705,039 | Gross: $83.47M

 Watch on Prime Video
rent/buy from $3.99
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 2
Oscar Nominations: 8
BAFTA Awards: 2
BAFTA Nominations: 8
Golden Globes: 3
Golden Globe Nominations: 4

Amadeus
32. Amadeus (1984)
R | 160 min | Biography, Drama, Music

 8.4  Rate 87 Metascore
The life, success and troubles of Wolfgang Amadeus Mozart, as told by Antonio Salieri, the contemporaneous composer who was deeply jealous of Mozart's talent and claimed to have murdered him.

Director: Milos Forman | Stars: F. Murray Abraham, Tom Hulce, Elizabeth Berridge, Roy Dotrice

Votes: 424,857 | Gross: $51.97M

 Watch on Prime Video
rent/buy from $3.79
*****
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 8
Oscar Nominations: 11
BAFTA Awards: 4
BAFTA Nominations: 9
Golden Globes: 4
Golden Globe Nominations: 6

The Lord of the Rings: The Return of the King
33. The Lord of the Rings: The Return of the King (2003)
PG-13 | 201 min | Action, Adventure, Drama

 9  Rate 94 Metascore
Gandalf and Aragorn lead the World of Men against Sauron's army to draw his gaze from Frodo and Sam as they approach Mount Doom with the One Ring.

Director: Peter Jackson | Stars: Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom

Votes: 1,961,651 | Gross: $377.85M

 Watch on Max
with Prime Video Channels
Actors: 4.8 Stars
Direction: 5 Stars
Screenplay: 4.9 Stars

Oscars: 11
Oscar Nominations: 11
BAFTA Awards: 4
BAFTA Nominations: 10
Golden Globes: 4
Golden Globe Nominations: 4

Gladiator
34. Gladiator (2000)
R | 155 min | Action, Adventure, Drama

 8.5  Rate 67 Metascore
A former Roman General sets out to exact vengeance against the corrupt emperor who murdered his family and sent him into slavery.

Director: Ridley Scott | Stars: Russell Crowe, Joaquin Phoenix, Connie Nielsen, Oliver Reed

Votes: 1,605,513 | Gross: $187.71M

*****
Actors: 5 Stars
Direction: 4.7 Stars
Screenplay: 4.7 Stars

Oscars: 5
Oscar Nominations: 12
BAFTA Awards: 4
BAFTA Nominations: 12
Golden Globes: 2
Golden Globe Nominations: 5

Titanic
35. Titanic (1997)
PG-13 | 194 min | Drama, Romance

 7.9  Rate 75 Metascore
A seventeen-year-old aristocrat falls in love with a kind but poor artist aboard the luxurious, ill-fated R.M.S. Titanic.

Director: James Cameron | Stars: Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates

Votes: 1,270,992 | Gross: $659.33M

 Watch on Paramount+
with Prime Video Channels
*****
Actors: 4.7 Stars
Direction: 4.9 Stars
Screenplay: 4.5 Stars

Oscars: 11
Oscar Nominations: 14
BAFTA Awards: 0
BAFTA Nominations: 8
Golden Globes: 4
Golden Globe Nominations: 8

From Here to Eternity
36. From Here to Eternity (1953)
Passed | 118 min | Drama, Romance, War

 7.6  Rate 85 Metascore
At a U.S. Army base in 1941 Hawaii, a private is cruelly punished for not boxing on his unit's team, while his commanding officer's wife and top aide begin a tentative affair.

Director: Fred Zinnemann | Stars: Burt Lancaster, Montgomery Clift, Deborah Kerr, Donna Reed

Votes: 50,533 | Gross: $30.50M

 Watch on Prime Video
rent/buy from $3.59
Actors: 4.8 Stars
Direction: 5 Stars
Screenplay: 4.8 Stars

Oscars: 8
Oscar Nominations: 13
BAFTA Awards: 0
BAFTA Nominations: 1
Golden Globes: 2
Golden Globe Nominations: 2

Saving Private Ryan
37. Saving Private Ryan (1998)
R | 169 min | Drama, War

 8.6  Rate 91 Metascore
Following the Normandy Landings, a group of U.S. soldiers go behind enemy lines to retrieve a paratrooper whose brothers have been killed in action.

Director: Steven Spielberg | Stars: Tom Hanks, Matt Damon, Tom Sizemore, Edward Burns

Votes: 1,484,773 | Gross: $216.54M

 Watch on Paramount+
with Prime Video Channels
Actors: 4.9 Stars
Direction: 4.9 Stars
Screenplay: 4.9 Stars

Oscars: 5
Oscar Nominations: 11
BAFTA Awards: 2
BAFTA Nominations: 8
Golden Globes: 2
Golden Globe Nominations: 5

Unforgiven
38. Unforgiven (1992)
R | 130 min | Drama, Western

 8.2  Rate 85 Metascore
Retired Old West gunslinger William Munny reluctantly takes on one last job, with the help of his old partner Ned Logan and a young man, The "Schofield Kid."

Director: Clint Eastwood | Stars: Clint Eastwood, Gene Hackman, Morgan Freeman, Richard Harris

Votes: 433,536 | Gross: $101.16M

 Watch on Prime Video
rent/buy from $3.99
Actors: 4.8 Stars
Direction: 4.9 Stars
Screenplay: 4.9 Stars

Oscars: 4
Oscar Nominations: 9
BAFTA Awards: 1
BAFTA Nominations: 6
Golden Globes: 2
Golden Globe Nominations: 4

Raiders of the Lost Ark
39. Raiders of the Lost Ark (1981)
PG | 115 min | Action, Adventure

 8.4  Rate 86 Metascore
In 1936, archaeologist and adventurer Indiana Jones is hired by the U.S. government to find the Ark of the Covenant before the Nazis can obtain its awesome powers.

Director: Steven Spielberg | Stars: Harrison Ford, Karen Allen, Paul Freeman, John Rhys-Davies

Votes: 1,031,102 | Gross: $248.16M

 Watch on Paramount+
with Prime Video Channels
Actors: 4.7 Stars
Direction: 4.9 Stars
Screenplay: 4.8 Stars

Oscars: 4
Oscar Nominations: 8
BAFTA Awards: 1
BAFTA Nominations: 6
Golden Globes: 0
Golden Globe Nominations: 1

Rocky
40. Rocky (1976)
PG | 120 min | Drama, Sport

 8.1  Rate 70 Metascore
A small-time Philadelphia boxer gets a supremely rare chance to fight the world heavyweight champion in a bout in which he strives to go the distance for his self-respect.

Director: John G. Avildsen | Stars: Sylvester Stallone, Talia Shire, Burt Young, Carl Weathers

Votes: 623,767 | Gross: $117.24M

 Watch on Max
with Prime Video Channels
Actors: 4.6 Stars
Direction: 4.8 Stars
Screenplay: 4.6 Stars

Oscars: 3
Oscar Nominations: 10
BAFTA Awards: 0
BAFTA Nominations: 5
Golden Globes: 1
Golden Globe Nominations: 6

A Streetcar Named Desire
41. A Streetcar Named Desire (1951)
PG | 122 min | Drama

 7.9  Rate 97 Metascore
Disturbed Blanche DuBois moves in with her sister in New Orleans and is tormented by her brutish brother-in-law while her reality crumbles around her.

Director: Elia Kazan | Stars: Vivien Leigh, Marlon Brando, Kim Hunter, Karl Malden

Votes: 113,747 | Gross: $8.00M

 Watch on Prime Video
rent/buy from $3.99
Actors: 4.9 Stars
Direction: 4.9 Stars
Screenplay: 4.9 Stars

Oscars: 4
Oscar Nominations: 12
BAFTA Awards: 1
BAFTA Nominations: 2
Golden Globes: 1
Golden Globe Nominations: 3

The Philadelphia Story
42. The Philadelphia Story (1940)
Not Rated | 112 min | Comedy, Romance

 7.9  Rate 96 Metascore
When a rich woman's ex-husband and a tabloid-type reporter turn up just before her planned remarriage, she begins to learn the truth about herself.

Director: George Cukor | Stars: Cary Grant, Katharine Hepburn, James Stewart, Ruth Hussey

Votes: 73,441

Actors: 4.9 Stars
Direction: 4.8 Stars
Screenplay: 4.8 Stars

Oscars: 2
Oscar Nominations: 6
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

To Kill a Mockingbird
43. To Kill a Mockingbird (1962)
Approved | 129 min | Crime, Drama

 8.3  Rate 88 Metascore
Atticus Finch, a widowed lawyer in Depression-era Alabama, defends a Black man against an undeserved rape charge, and tries to educate his young children against prejudice.

Director: Robert Mulligan | Stars: Gregory Peck, John Megna, Frank Overton, Rosemary Murphy

Votes: 331,075

 Watch on Plex
Go to plex.tv
Actors: 4.8 Stars
Direction: 4.8 Stars
Screenplay: 4.7 Stars

Oscars: 3
Oscar Nominations: 8
BAFTA Awards: 0
BAFTA Nominations: 2
Golden Globes: 3
Golden Globe Nominations: 5

An American in Paris
44. An American in Paris (1951)
Passed | 114 min | Drama, Musical, Romance

 7.2  Rate 83 Metascore
Three friends struggle to find work in Paris. Things become more complicated when two of them fall in love with the same woman.

Director: Vincente Minnelli | Stars: Gene Kelly, Leslie Caron, Oscar Levant, Georges Guétary

Votes: 36,604 | Gross: $4.50M

 Watch on Prime Video
rent/buy from $3.99
Actors: 4.8 Stars
Direction: 4.8 Stars
Screenplay: 4.9 Stars

Oscars: 6
Oscar Nominations: 8
BAFTA Awards: 0
BAFTA Nominations: 1
Golden Globes: 1
Golden Globe Nominations: 3

The Best Years of Our Lives
45. The Best Years of Our Lives (1946)
Approved | 170 min | Drama, Romance, War

 8.1  Rate 93 Metascore
Three World War II veterans, two of them traumatized or disabled, return home to the American midwest to discover that they and their families have been irreparably changed.

Director: William Wyler | Stars: Myrna Loy, Dana Andrews, Fredric March, Teresa Wright

Votes: 69,970 | Gross: $23.65M

 Watch on Freevee
Watch Free on Freevee
Actors: 4.8 Stars
Direction: 4.9 Stars
Screenplay: 4.9 Stars

Oscars: 7
Oscar Nominations: 8
BAFTA Awards: 1
BAFTA Nominations: 1
Golden Globes: 1
Golden Globe Nominations: 1

My Fair Lady
46. My Fair Lady (1964)
G | 170 min | Drama, Family, Musical

 7.7  Rate 95 Metascore
In 1910s London, snobbish phonetics professor Henry Higgins agrees to a wager that he can make crude flower girl, Eliza Doolittle, presentable in high society.

Director: George Cukor | Stars: Audrey Hepburn, Rex Harrison, Stanley Holloway, Wilfrid Hyde-White

Votes: 101,244 | Gross: $72.00M

 Watch on Paramount+
with Prime Video Channels
Actors: 4.9 Stars
Direction: 4.9 Stars
Screenplay: 4.8 Stars

Oscars: 8
Oscar Nominations: 12
BAFTA Awards: 1
BAFTA Nominations: 2
Golden Globes: 3
Golden Globe Nominations: 5

A Clockwork Orange
47. A Clockwork Orange (1971)
R | 136 min | Crime, Sci-Fi

 8.3  Rate 77 Metascore
In the future, a sadistic gang leader is imprisoned and volunteers for a conduct-aversion experiment, but it doesn't go as planned.

Director: Stanley Kubrick | Stars: Malcolm McDowell, Patrick Magee, Michael Bates, Warren Clarke

Votes: 875,966 | Gross: $6.21M

Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 0
Oscar Nominations: 4
BAFTA Awards: 0
BAFTA Nominations: 7
Golden Globes: 0
Golden Globe Nominations: 3

Doctor Zhivago
48. Doctor Zhivago (1965)
PG-13 | 197 min | Drama, Romance, War

 7.9  Rate 69 Metascore
The life of a Russian physician and poet who, although married to another, falls in love with a political activist's wife and experiences hardship during World War I and then the October Revolution.

Director: David Lean | Stars: Omar Sharif, Julie Christie, Geraldine Chaplin, Rod Steiger

Votes: 81,605 | Gross: $111.72M

Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 5
Oscar Nominations: 10
BAFTA Awards: 0
BAFTA Nominations: 3
Golden Globes: 5
Golden Globe Nominations: 6

The Searchers
49. The Searchers (1956)
Passed | 119 min | Adventure, Drama, Western

 7.8  Rate 94 Metascore
An American Civil War veteran embarks on a years-long journey to rescue his niece from the Comanches after the rest of his brother's family is massacred in a raid on their Texas farm.

Director: John Ford | Stars: John Wayne, Jeffrey Hunter, Vera Miles, Ward Bond

Votes: 95,614

*****
Actors: 4.9 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 0
Oscar Nominations: 0
BAFTA Awards: 0
BAFTA Award Nominations: 0
Golden Globes: 0
Golden Globe Nominations: 0

Jaws
50. Jaws (1975)
PG | 124 min | Adventure, Mystery, Thriller

 8.1  Rate 87 Metascore
When a killer shark unleashes chaos on a beach community off Cape Cod, it's up to a local sheriff, a marine biologist, and an old seafarer to hunt the beast down.

Director: Steven Spielberg | Stars: Roy Scheider, Robert Shaw, Richard Dreyfuss, Lorraine Gary

Votes: 654,563 | Gross: $260.00M

Actors: 4.7 Stars
Direction: 4.8 Stars
Screenplay: 4.7 Stars

Oscars: 3
Oscar Nominations: 4
BAFTA Awards: 0
BAFTA Nominations: 6
Golden Globes: 1
Golden Globe Nominations: 4

Patton
51. Patton (1970)
GP | 172 min | Biography, Drama, War

 7.9  Rate 86 Metascore
The World War II phase of the career of controversial American general George S. Patton.

Director: Franklin J. Schaffner | Stars: George C. Scott, Karl Malden, Stephen Young, Michael Strong

Votes: 107,432 | Gross: $61.70M

*****
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 7
Oscar Nominations: 10
BAFTA Awards: 0
BAFTA Nominations: 2
Golden Globes: 1
Golden Globe Nominations: 3

Butch Cassidy and the Sundance Kid
52. Butch Cassidy and the Sundance Kid (1969)
PG | 110 min | Biography, Crime, Drama

 8  Rate 66 Metascore
In 1890s Wyoming, Butch Cassidy and The Sundance Kid lead a band of outlaws. When a train robbery goes wrong, they find themselves on the run with a posse hard on their heels. After considering their options, they escape to South America.

Director: George Roy Hill | Stars: Paul Newman, Robert Redford, Katharine Ross, Strother Martin

Votes: 225,808 | Gross: $102.31M

*****
Actors: 4.6 Stars
Direction: 4.8 Stars
Screenplay: 4.7 Stars

Oscars: 4
Oscar Nominations: 7
BAFTA Awards: 8
BAFTA Nominations: 9
Golden Globes: 1
Golden Globe Nominations: 4

The Treasure of the Sierra Madre
53. The Treasure of the Sierra Madre (1948)
Passed | 126 min | Adventure, Drama, Western

 8.2  Rate 98 Metascore
Two down-on-their-luck Americans searching for work in 1920s Mexico convince an old prospector to help them mine for gold in the Sierra Madre Mountains.

Director: John Huston | Stars: Humphrey Bogart, Walter Huston, Tim Holt, Bruce Bennett

Votes: 131,688 | Gross: $5.01M

Actors: 4.7 Stars
Direction: 4.8 Stars
Screenplay: 4.8 Stars

Oscars: 3
Oscar Nominations: 4
BAFTA Awards: 0
BAFTA Nominations: 1
Golden Globes: 3
Golden Globe Nominations: 3

The Good, the Bad and the Ugly
54. The Good, the Bad and the Ugly (1966)
Approved | 178 min | Adventure, Western

 8.8  Rate 90 Metascore
A bounty hunting scam joins two men in an uneasy alliance against a third in a race to find a fortune in gold buried in a remote cemetery.

Director: Sergio Leone | Stars: Clint Eastwood, Eli Wallach, Lee Van Cleef, Aldo Giuffrè

Votes: 805,279 | Gross: $6.10M

Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 0
Oscar Nominations: 0
BAFTA Awards: 0
BAFTA Award Nominations: 0
Golden Globes: 0
Golden Globe Nominations: 0

The Apartment
55. The Apartment (1960)
Approved | 125 min | Comedy, Drama, Romance

 8.3  Rate 94 Metascore
A Manhattan insurance clerk tries to rise in his company by letting its executives use his apartment for trysts, but complications and a romance of his own ensue.

Director: Billy Wilder | Stars: Jack Lemmon, Shirley MacLaine, Fred MacMurray, Ray Walston

Votes: 195,147 | Gross: $18.60M

Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 5
Oscar Nominations: 10
BAFTA Awards: 3
BAFTA Nominations: 3
Golden Globes: 3
Golden Globe Nominations: 4

Platoon
56. Platoon (1986)
R | 120 min | Drama, War

 8.1  Rate 92 Metascore
Chris Taylor, a neophyte recruit in Vietnam, finds himself caught in a battle of wills between two sergeants, one good and the other evil. A shrewd examination of the brutality of war and the duality of man in conflict.

Director: Oliver Stone | Stars: Charlie Sheen, Tom Berenger, Willem Dafoe, Keith David

Votes: 436,540 | Gross: $138.53M

*****
Actors: 4.8 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 4
Oscar Nominations: 8
BAFTA Awards: 2
BAFTA Nominations: 3
Golden Globes: 3
Golden Globe Nominations: 4

High Noon
57. High Noon (1952)
PG | 85 min | Drama, Thriller, Western

 8  Rate 89 Metascore
A town Marshal, despite the disagreements of his newlywed bride and the townspeople around him, must face a gang of deadly killers alone at "high noon" when the gang leader, an outlaw he "sent up" years ago, arrives on the noon train.

Director: Fred Zinnemann | Stars: Gary Cooper, Grace Kelly, Thomas Mitchell, Lloyd Bridges

Votes: 109,633 | Gross: $9.45M

*****
Actors: 4.6 Stars
Direction: 4.7 Stars
Screenplay: 4.7 Stars

Oscars: 4
Oscar Nominations: 7
BAFTA Awards: 0
BAFTA Nominations: 0
Golden Globes: 4
Golden Globe Nominations: 7

Braveheart
58. Braveheart (1995)
R | 178 min | Biography, Drama, History

 8.3  Rate 68 Metascore
Scottish warrior William Wallace leads his countrymen in a rebellion to free his homeland from the tyranny of King Edward I of England.

Director: Mel Gibson | Stars: Mel Gibson, Sophie Marceau, Patrick McGoohan, Angus Macfadyen

Votes: 1,085,439 | Gross: $75.60M

Actors: 4.7 Stars
Direction: 4.8 Stars
Screenplay: 4.7 Stars

Oscars: 5
Oscar Nominations: 10
BAFTA Awards: 3
BAFTA Nominations: 5
Golden Globes: 1
Golden Globe Nominations: 4

Dances with Wolves
59. Dances with Wolves (1990)
PG-13 | 181 min | Adventure, Drama, Western

 8  Rate 72 Metascore
Lieutenant John Dunbar, assigned to a remote western Civil War outpost, finds himself engaging with a neighbouring Sioux settlement, causing him to question his own purpose.

Director: Kevin Costner | Stars: Kevin Costner, Mary McDonnell, Graham Greene, Rodney A. Grant

Votes: 288,066 | Gross: $184.21M

*****
Actors: 4.5 Stars
Direction: 4.8 Stars
Screenplay: 4.6 Stars

Oscars: 7
Oscar Nominations: 12
BAFTA Awards: 0
BAFTA Nominations: 9
Golden Globes: 3
Golden Globe Nominations: 6

Jurassic Park
60. Jurassic Park (1993)
PG-13 | 127 min | Action, Adventure, Sci-Fi

 8.2  Rate 68 Metascore
A pragmatic paleontologist touring an almost complete theme park on an island in Central America is tasked with protecting a couple of kids after a power failure causes the park's cloned dinosaurs to run loose.

Director: Steven Spielberg | Stars: Sam Neill, Laura Dern, Jeff Goldblum, Richard Attenborough

Votes: 1,058,419 | Gross: $402.45M

Actors: 4.6 Stars
Direction: 4.9 Stars
Screenplay: 4.7 Stars

Oscars: 3
Oscar Nominations: 3
BAFTA Awards: 1
BAFTA Nominations: 2
Golden Globes: 0
Golden Globe Nominations: 0

The Exorcist
61. The Exorcist (1973)
R | 122 min | Horror

 8.1  Rate 82 Metascore
When a young girl is possessed by a mysterious entity, her mother seeks the help of two Catholic priests to save her life.

Director: William Friedkin | Stars: Ellen Burstyn, Max von Sydow, Linda Blair, Lee J. Cobb

Votes: 451,502 | Gross: $232.91M

Actors: 4.8 Stars
Direction: 4.8 Stars
Screenplay: 4.8 Stars

Oscars: 2
Oscar Nominations: 10
BAFTA Awards: 0
BAFTA Nominations: 1
Golden Globes: 4
Golden Globe Nominations: 7

The Pianist
62. The Pianist (2002)
R | 150 min | Biography, Drama, Music

 8.5  Rate 85 Metascore
During WWII, acclaimed Polish musician Wladyslaw faces various struggles as he loses contact with his family. As the situation worsens, he hides in the ruins of Warsaw in order to survive.

Director: Roman Polanski | Stars: Adrien Brody, Thomas Kretschmann, Frank Finlay, Emilia Fox

Votes: 903,558 | Gross: $32.57M

*****
Actors: 4.9 Stars
Direction: 4.8 Stars
Screenplay: 4.7 Stars

Oscars: 3
Oscar Nominations: 7
BAFTA Awards: 1
BAFTA Nominations: 5
Golden Globes: 0
Golden Globe Nominations: 2

Goodfellas
63. Goodfellas (1990)
R | 145 min | Biography, Crime, Drama

 8.7  Rate 92 Metascore
The story of Henry Hill and his life in the mafia, covering his relationship with his wife Karen and his mob partners Jimmy Conway and Tommy DeVito.

Director: Martin Scorsese | Stars: Robert De Niro, Ray Liotta, Joe Pesci, Lorraine Bracco

Votes: 1,246,267 | Gross: $46.84M

*****
Actors: 4.8 Stars
Direction: 4.9 Stars
Screenplay: 5 Stars

Oscars: 1
Oscar Nominations: 6
BAFTA Awards: 5
BAFTA Nominations: 7
Golden Globes: 0
Golden Globe Nominations: 5

The Deer Hunter
64. The Deer Hunter (1978)
R | 183 min | Drama, War

 8.1  Rate 89 Metascore
An in-depth examination of the ways in which the Vietnam War impacts and disrupts the lives of several friends in a small steel mill town in Pennsylvania.

Director: Michael Cimino | Stars: Robert De Niro, Christopher Walken, John Cazale, John Savage

Votes: 359,983 | Gross: $48.98M

Actors: 5 Stars
Direction: 4.8 Stars
Screenplay: 4.9 Stars

Oscars: 5
Oscar Nominations: 9
BAFTA Awards: 2
BAFTA Nominations: 9
Golden Globes: 1
Golden Globe Nominations: 6

All Quiet on the Western Front
65. All Quiet on the Western Front (1930)
Passed | 152 min | Drama, War

 8.1  Rate 91 Metascore
A German youth eagerly enters World War I, but his enthusiasm wanes as he gets a firsthand view of the horror.

Director: Lewis Milestone | Stars: Lew Ayres, Louis Wolheim, John Wray, Arnold Lucy

Votes: 67,287 | Gross: $3.27M

*****
Actors: 4.7 Stars
Direction: 4.9 Stars
Screenplay: 4.7 Stars

Oscars: 2
Oscar Nominations: 4
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

Bonnie and Clyde
66. Bonnie and Clyde (1967)
R | 111 min | Action, Biography, Crime

 7.7  Rate 86 Metascore
Bored waitress Bonnie Parker falls in love with an ex-con named Clyde Barrow and together they start a violent crime spree through the country, stealing cars and robbing banks.

Director: Arthur Penn | Stars: Warren Beatty, Faye Dunaway, Michael J. Pollard, Gene Hackman

Votes: 119,963

Actors: 4.8 Stars
Direction: 4.7 Stars
Screenplay: 4.8 Stars

Oscars: 2
Oscar Nominations: 10
BAFTA Awards: 2
BAFTA Nominations: 4
Golden Globes: 0
Golden Globe Nominations: 7

The French Connection
67. The French Connection (1971)
R | 104 min | Action, Crime, Drama

 7.7  Rate 94 Metascore
A pair of NYPD detectives in the Narcotics Bureau stumble onto a heroin smuggling ring based in Marseilles, but stopping them and capturing their leaders proves an elusive goal.

Director: William Friedkin | Stars: Gene Hackman, Roy Scheider, Fernando Rey, Tony Lo Bianco

Votes: 134,526 | Gross: $15.63M

 Watch on Max
with Prime Video Channels
*****
Actors: 4.9 Stars
Direction: 4.8 Stars
Screenplay: 4.8 Stars

Oscars: 5
Oscar Nominations: 8
BAFTA Awards: 2
BAFTA Nominations: 5
Golden Globes: 3
Golden Globe Nominations: 4

City Lights
68. City Lights (1931)
G | 87 min | Comedy, Drama, Romance

 8.5  Rate 99 Metascore
With the aid of a wealthy erratic tippler, a dewy-eyed tramp who has fallen in love with a sightless flower girl accumulates money to be able to help her medically.

Director: Charles Chaplin | Stars: Charles Chaplin, Virginia Cherrill, Florence Lee, Harry Myers

Votes: 194,802 | Gross: $0.02M

 Watch on Max
with Prime Video Channels
*****
Actors: 4.9 Stars
Direction: 4.9 Stars
Screenplay: 4.8 Stars

Oscars: 0
Oscar Nominations: 0
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

It Happened One Night
69. It Happened One Night (1934)
Passed | 105 min | Comedy, Romance

 8.1  Rate 87 Metascore
A renegade reporter trailing a young runaway heiress for a big story joins her on a bus heading from Florida to New York, and they end up stuck with each other when the bus leaves them behind at one of the stops.

Director: Frank Capra | Stars: Clark Gable, Claudette Colbert, Walter Connolly, Roscoe Karns

Votes: 111,458 | Gross: $4.36M

 Watch on Prime Video
rent/buy from $2.99
*****
Actors: 4.7 Stars
Direction: 4.8 Stars
Screenplay: 4.8 Stars

Oscars: 5
Oscar Nominations: 5
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

A Place in the Sun
70. A Place in the Sun (1951)
Passed | 122 min | Drama, Romance

 7.7  Rate 76 Metascore
A poor boy gets a job working for his rich uncle and ends up falling in love with two women.

Director: George Stevens | Stars: Montgomery Clift, Elizabeth Taylor, Shelley Winters, Anne Revere

Votes: 24,590

 Watch on Pluto TV
Go to pluto.tv
*****
Actors: 4.6 Stars
Direction: 4.8 Stars
Screenplay: 4.7 Stars

Oscars: 6
Oscar Nominations: 9
BAFTA Awards: 0
BAFTA Nominations: 0
Golden Globes: 1
Golden Globe Nominations: 4

Midnight Cowboy
71. Midnight Cowboy (1969)
R | 113 min | Drama

 7.8  Rate 79 Metascore
A naive hustler travels from Texas to New York City to seek personal fortune, finding a new friend in the process.

Director: John Schlesinger | Stars: Dustin Hoffman, Jon Voight, Sylvia Miles, John McGiver

Votes: 119,571 | Gross: $44.79M

 Watch on Pluto TV
Go to pluto.tv
*****
Actors: 4.8 Stars
Direction: 4.7 Stars
Screenplay: 4.9 Stars

Oscars: 3
Oscar Nominations: 7
BAFTA Awards: 6
BAFTA Nominations: 6
Golden Globes: 1
Golden Globe Nominations: 7

Mr. Smith Goes to Washington
72. Mr. Smith Goes to Washington (1939)
Passed | 129 min | Comedy, Drama

 8.1  Rate 73 Metascore
A naive youth leader is appointed to fill a vacancy in the U.S. Senate. His idealistic plans promptly collide with corruption at home and subterfuge from his hero in Washington, but he tries to forge ahead despite attacks on his character.

Director: Frank Capra | Stars: James Stewart, Jean Arthur, Claude Rains, Edward Arnold

Votes: 120,808 | Gross: $9.60M

Actors: 4.8 Stars
Direction: 4.8 Stars
Screenplay: 4.8 Stars

Oscars: 1
Oscar Nominations: 11
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

Rain Man
73. Rain Man (1988)
R | 133 min | Drama

 8  Rate 65 Metascore
After a selfish L.A. yuppie learns his estranged father left a fortune to an autistic-savant brother in Ohio that he didn't know existed, he absconds with his brother and sets out across the country, hoping to gain a larger inheritance.

Director: Barry Levinson | Stars: Dustin Hoffman, Tom Cruise, Valeria Golino, Gerald R. Molen

Votes: 543,182 | Gross: $178.80M

 Watch on MGM+
with Prime Video Channels
Actors: 5 Stars
Direction: 4.9 Stars
Screenplay: 4.8 Stars

Oscars: 4
Oscar Nominations: 8
BAFTA Awards: 0
BAFTA Nominations: 3
Golden Globes: 2
Golden Globe Nominations: 4

Annie Hall
74. Annie Hall (1977)
PG | 93 min | Comedy, Romance

 8  Rate 92 Metascore
Alvy Singer, a divorced Jewish comedian, reflects on his relationship with ex-lover Annie Hall, an aspiring nightclub singer, which ended abruptly just like his previous marriages.

Director: Woody Allen | Stars: Woody Allen, Diane Keaton, Tony Roberts, Carol Kane

Votes: 276,877 | Gross: $39.20M

 Watch on Prime Video
included with Prime
*****
Actors: 4.8 Stars
Direction: 5 Stars
Screenplay: 4.9 Stars

Oscars: 4
Oscar Nominations: 5
BAFTA Awards: 5
BAFTA Nominations: 6
Golden Globes: 1
Golden Globe Nominations: 5

Fargo
75. Fargo (1996)
R | 98 min | Crime, Thriller

 8.1  Rate 87 Metascore
Minnesota car salesman Jerry Lundegaard's inept crime falls apart due to his and his henchmen's bungling and the persistent police work of the quite pregnant Marge Gunderson.

Directors: Joel Coen, Ethan Coen | Stars: William H. Macy, Frances McDormand, Steve Buscemi, Peter Stormare

Votes: 721,107 | Gross: $24.61M

 Watch on Cinemax
with Prime Video Channels
Actors: 5 Stars
Direction: 5 Stars
Screenplay: 5 Stars

Oscars: 2
Oscar Nominations: 7
BAFTA Awards: 1
BAFTA Nominations: 6
Golden Globes: 0
Golden Globe Nominations: 4

Giant
76. Giant (1956)
G | 201 min | Drama, Western

 7.6  Rate 84 Metascore
Sprawling epic covering the life of a Texas cattle rancher and his family and associates.

Director: George Stevens | Stars: Elizabeth Taylor, Rock Hudson, James Dean, Carroll Baker

Votes: 41,978

 Watch on Prime Video
rent/buy from $3.99
*****
Actors: 4.8 Stars
Direction: 4.7 Stars
Screenplay: 4.8 Stars

Oscars: 1
Oscar Nominations: 10
BAFTA Awards: 0
BAFTA Nominations: 0
Golden Globes: 0
Golden Globe Nominations: 1

Shane
77. Shane (1953)
Not Rated | 118 min | Drama, Western

 7.6  Rate 85 Metascore
A weary gunfighter in 1880s Wyoming begins to envision a quieter life after befriending a homestead family with a young son who idolizes him, but a smoldering range war forces him to act.

Director: George Stevens | Stars: Alan Ladd, Jean Arthur, Van Heflin, Brandon De Wilde

Votes: 43,872 | Gross: $20.00M

 Watch on Prime Video
rent/buy from $3.89
Actors: 4.7 Stars
Direction: 4.8 Stars
Screenplay: 4.7 Stars

Oscars: 1
Oscar Nominations: 6
BAFTA Awards: 0
BAFTA Nominations: 2
Golden Globes: 0
Golden Globe Nominations: 0

The Grapes of Wrath
78. The Grapes of Wrath (1940)
Passed | 129 min | Drama

 8.1  Rate 96 Metascore
An Oklahoma family, driven off their farm by the poverty and hopelessness of the Dust Bowl, joins the westward migration to California, suffering the misfortunes of the homeless in the Great Depression.

Director: John Ford | Stars: Henry Fonda, Jane Darwell, John Carradine, Charley Grapewin

Votes: 99,336 | Gross: $0.06M

 Watch on Prime Video
rent/buy from $3.79
Actors: 4.6 Stars
Direction: 4.8 Stars
Screenplay: 4.7 Stars

Oscars: 2
Oscar Nominations: 7
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

The Green Mile
79. The Green Mile (1999)
R | 189 min | Crime, Drama, Fantasy

 8.6  Rate 61 Metascore
A tale set on death row, where gentle giant John Coffey possesses the mysterious power to heal people's ailments. When the lead guard, Paul Edgecombe, recognizes John's gift, he tries to help stave off the condemned man's execution.

Director: Frank Darabont | Stars: Tom Hanks, Michael Clarke Duncan, David Morse, Bonnie Hunt

Votes: 1,395,202 | Gross: $136.80M

Actors: 4.8 Stars
Direction: 4.9 Stars
Screenplay: 4.9 Stars

Oscars: 0
Oscar Nominations: 4
BAFTA Awards: 0
BAFTA Nominations: 0
Golden Globes: 0
Golden Globe Nominations: 1

Close Encounters of the Third Kind
80. Close Encounters of the Third Kind (1977)
PG | 138 min | Drama, Sci-Fi

 7.6  Rate 90 Metascore
Roy Neary, an Indiana electric lineman, finds his quiet and ordinary daily life turned upside down after a close encounter with a UFO, spurring him to an obsessed cross-country quest for answers as a momentous event approaches.

Director: Steven Spielberg | Stars: Richard Dreyfuss, François Truffaut, Teri Garr, Melinda Dillon

Votes: 215,380 | Gross: $132.09M

 Watch on Prime Video
rent/buy from $3.59
Actors: 4.6 Stars
Direction: 4.9 Stars
Screenplay: 4.7 Stars

Oscars: 1
Oscar Nominations: 8
BAFTA Awards: 0
BAFTA Nominations: 4
Golden Globes: 1
Golden Globe Nominations: 8

Nashville
81. Nashville (1975)
R | 160 min | Comedy, Drama, Music

 7.6  Rate 96 Metascore
Over the course of a few hectic days, numerous interrelated people prepare for a political convention.

Director: Robert Altman | Stars: Keith Carradine, Karen Black, Ronee Blakley, Shelley Duvall

Votes: 28,342 | Gross: $14.82M

*****
Actors: 4.7 Stars
Direction: 4.8 Stars
Screenplay: 4.8 Stars

Oscars: 1
Oscar Nominations: 5
BAFTA Awards: 1
BAFTA Nominations: 5
Golden Globes: 1
Golden Globe Nominations: 11

Network
82. Network (1976)
R | 121 min | Drama

 8.1  Rate 83 Metascore
A television network cynically exploits a deranged former anchor's ravings and revelations about the news media for its own profit, but finds that his message may be difficult to control.

Director: Sidney Lumet | Stars: Faye Dunaway, William Holden, Peter Finch, Robert Duvall

Votes: 169,319

 Watch on Prime Video
rent/buy from $3.99
Actors: 4.8 Stars
Direction: 4.9 Stars
Screenplay: 4.9 Stars

Oscars: 4
Oscar Nominations: 10
BAFTA Awards: 1
BAFTA Nominations: 9
Golden Globes: 4
Golden Globe Nominations: 5

The Graduate
83. The Graduate (1967)
PG | 106 min | Comedy, Drama, Romance

 8  Rate 83 Metascore
A disillusioned college graduate finds himself torn between his older lover and her daughter.

Director: Mike Nichols | Stars: Dustin Hoffman, Anne Bancroft, Katharine Ross, William Daniels

Votes: 287,189 | Gross: $104.95M

 Watch on Prime Video
rent from $3.99
Actors: 5 Stars
Direction: 4.8 Stars
Screenplay: 4.8 Stars

Oscars: 1
Oscar Nominations: 7
BAFTA Awards: 5
BAFTA Nominations: 7
Golden Globes: 5
Golden Globe Nominations: 7

American Graffiti
84. American Graffiti (1973)
PG | 110 min | Comedy, Drama

 7.4  Rate 97 Metascore
A group of teenagers in California's central valley spend one final night after their 1962 high school graduation cruising the strip with their buddies before they pursue their varying goals.

Director: George Lucas | Stars: Richard Dreyfuss, Ron Howard, Paul Le Mat, Charles Martin Smith

Votes: 97,507 | Gross: $115.00M

 Watch on Prime Video
rent/buy from $3.99
*****
Actors: 4.7 Stars
Direction: 4.8 Stars
Screenplay: 4.8 Stars

Oscars: 0
Oscar Nominations: 5
BAFTA Awards: 0
BAFTA Nominations: 1
Golden Globes: 2
Golden Globe Nominations: 4

Pulp Fiction
85. Pulp Fiction (1994)
R | 154 min | Crime, Drama

 8.9  Rate 95 Metascore
The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.

Director: Quentin Tarantino | Stars: John Travolta, Uma Thurman, Samuel L. Jackson, Bruce Willis

Votes: 2,199,399 | Gross: $107.93M

 Watch on Max
with Prime Video Channels
Actors: 4.7 Stars
Direction: 4.9 Stars
Screenplay: 5 Stars

Oscars: 1
Oscar Nominations: 7
BAFTA Awards: 2
BAFTA Nominations: 8
Golden Globes: 1
Golden Globe Nominations: 6

Terms of Endearment
86. Terms of Endearment (1983)
PG | 132 min | Comedy, Drama

 7.4  Rate 79 Metascore
Follows hard-to-please Aurora looking for love and her daughter's family problems.

Director: James L. Brooks | Stars: Shirley MacLaine, Debra Winger, Jack Nicholson, Danny DeVito

Votes: 64,988 | Gross: $108.42M

 Watch on Paramount+
with subscription
*****
Actors: 5 Stars
Direction: 4.9 Stars
Screenplay: 4.9 Stars

Oscars: 5
Oscar Nominations: 11
BAFTA Awards: 0
BAFTA Nominations: 1
Golden Globes: 4
Golden Globe Nominations: 6

Good Will Hunting
87. Good Will Hunting (1997)
R | 126 min | Drama, Romance

 8.3  Rate 71 Metascore
Will Hunting, a janitor at M.I.T., has a gift for mathematics, but needs help from a psychologist to find direction in his life.

Director: Gus Van Sant | Stars: Robin Williams, Matt Damon, Ben Affleck, Stellan Skarsgård

Votes: 1,055,616 | Gross: $138.43M

Actors: 4.8 Stars
Direction: 4.9 Stars
Screenplay: 5 Stars

Oscars: 2
Oscar Nominations: 9
BAFTA Awards: 0
BAFTA Nominations: 0
Golden Globes: 1
Golden Globe Nominations: 4

The African Queen
88. The African Queen (1951)
PG | 105 min | Adventure, Drama, Romance

 7.7  Rate 91 Metascore
In WWI East Africa, a gin-swilling Canadian riverboat captain is persuaded by a strait-laced English missionary to undertake a trip up a treacherous river and use his boat to attack a German gunship.

Director: John Huston | Stars: Humphrey Bogart, Katharine Hepburn, Robert Morley, Peter Bull

Votes: 83,880 | Gross: $0.54M

 Watch on Prime Video
included with Prime
Actors: 4.7 Stars
Direction: 4.8 Stars
Screenplay: 4.8 Stars

Oscars: 1
Oscar Nominations: 4
BAFTA Awards: 0
BAFTA Nominations: 3
Golden Globes: 0
Golden Globe Nominations: 0

Stagecoach
89. Stagecoach (1939)
Passed | 96 min | Adventure, Drama, Western

 7.8  Rate 93 Metascore
A group of people traveling on a stagecoach find their journey complicated by the threat of Geronimo and learn something about each other in the process.

Director: John Ford | Stars: John Wayne, Claire Trevor, Andy Devine, John Carradine

Votes: 53,327

 Watch on Freevee
Watch Free on Freevee
Actors: 4.6 Stars
Direction: 4.8 Stars
Screenplay: 4.7 Stars

Oscars: 2
Oscar Nominations: 7
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

Mutiny on the Bounty
90. Mutiny on the Bounty (1935)
Passed | 132 min | Adventure, Biography, Drama

 7.6  Rate 87 Metascore
First mate Fletcher Christian leads a revolt against his sadistic commander, Captain Bligh, in this classic seafaring adventure, based on the real-life 1789 mutiny.

Director: Frank Lloyd | Stars: Charles Laughton, Clark Gable, Franchot Tone, Herbert Mundin

Votes: 24,665

Actors: 4.9 Stars
Direction: 4.7 Stars
Screenplay: 4.8 Stars

Oscars: 1
Oscar Nominations: 8
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

The Great Dictator
91. The Great Dictator (1940)
G | 125 min | Comedy, Drama, War

 8.4  Rate
Dictator Adenoid Hynkel tries to expand his empire while a poor Jewish barber tries to avoid persecution from Hynkel's regime.

Director: Charles Chaplin | Stars: Charles Chaplin, Paulette Goddard, Jack Oakie, Reginald Gardiner

Votes: 235,987 | Gross: $0.29M

Actors: 5 Stars
Direction: 4.9 Stars
Screenplay: 4.8 Stars

Oscars: 0
Oscar Nominations: 5
BAFTA Awards: 0
BAFTA Nominations: 0
Golden Globes: 0
Golden Globe Nominations: 0

Double Indemnity
92. Double Indemnity (1944)
Passed | 107 min | Crime, Drama, Film-Noir

 8.3  Rate 95 Metascore
A Los Angeles insurance representative lets an alluring housewife seduce him into a scheme of insurance fraud and murder that arouses the suspicion of his colleague, an insurance investigator.

Director: Billy Wilder | Stars: Fred MacMurray, Barbara Stanwyck, Edward G. Robinson, Byron Barr

Votes: 166,410 | Gross: $5.72M

 Watch on Prime Video
rent/buy from $3.59
Actors: 4.8 Stars
Direction: 4.8 Stars
Screenplay: 4.8 Stars

Oscars: 0
Oscar Nominations: 7
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

The Maltese Falcon
93. The Maltese Falcon (1941)
Passed | 100 min | Crime, Film-Noir, Mystery

 8  Rate 97 Metascore
San Francisco private detective Sam Spade takes on a case that involves him with three eccentric criminals, a gorgeous liar and their quest for a priceless statuette, with the stakes rising after his partner is murdered.

Director: John Huston | Stars: Humphrey Bogart, Mary Astor, Gladys George, Peter Lorre

Votes: 165,820 | Gross: $2.11M

 Watch on Prime Video
rent/buy from $3.99
Actors: 4.7 Stars
Direction: 4.7 Stars
Screenplay: 4.8 Stars

Oscars: 0
Oscar Nominations: 3
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

Wuthering Heights
94. Wuthering Heights (1939)
Passed | 104 min | Drama, Romance

 7.5  Rate
A servant in the house of Wuthering Heights tells a traveler the unfortunate tale of lovers Cathy and Heathcliff.

Director: William Wyler | Stars: Merle Oberon, Laurence Olivier, David Niven, Flora Robson

Votes: 19,640 | Gross: $0.76M

*****
Actors: 4.8 Stars
Direction: 4.9 Stars
Screenplay: 4.8 Stars

Oscars: 1
Oscar Nominations: 8
BAFTA Awards: N/A
BAFTA Nominations: N/A
Golden Globes: N/A
Golden Globe Nominations: N/A

Taxi Driver
95. Taxi Driver (1976)
R | 114 min | Crime, Drama

 8.2  Rate 94 Metascore
A mentally unstable veteran works as a nighttime taxi driver in New York City, where the perceived decadence and sleaze fuels his urge for violent action.

Director: Martin Scorsese | Stars: Robert De Niro, Jodie Foster, Cybill Shepherd, Albert Brooks

Votes: 912,135 | Gross: $28.26M

Actors: 5 Stars
Direction: 5 Stars
Screenplay: 4.8 Stars

Oscars: 0
Oscar Nominations: 4
BAFTA Awards: 2
BAFTA Nominations: 6
Golden Globes: 0
Golden Globe Nominations: 2

Rear Window
96. Rear Window (1954)
PG | 112 min | Mystery, Thriller

 8.5  Rate 100 Metascore
A photographer in a wheelchair spies on his neighbors from his Greenwich Village courtyard apartment window, and becomes convinced one of them has committed murder, despite the skepticism of his fashion-model girlfriend.

Director: Alfred Hitchcock | Stars: James Stewart, Grace Kelly, Wendell Corey, Thelma Ritter

Votes: 518,851 | Gross: $36.76M

Actors: 4.8 Stars
Direction: 5 Stars
Screenplay: 4.9 Stars

Oscars: 0
Oscar Nominations: 4
BAFTA Awards: 0
BAFTA Nominations: 1
Golden Globes: 0
Golden Globe Nominations: 0

The Third Man
97. The Third Man (1949)
Approved | 93 min | Film-Noir, Mystery, Thriller

 8.1  Rate 97 Metascore
Pulp novelist Holly Martins travels to shadowy, postwar Vienna, only to find himself investigating the mysterious death of an old friend, Harry Lime.

Director: Carol Reed | Stars: Orson Welles, Joseph Cotten, Alida Valli, Trevor Howard

Votes: 180,674 | Gross: $0.45M

Actors: 4.8 Stars
Direction: 4.9 Stars
Screenplay: 4.8 Stars

Oscars: 1
Oscar Nominations: 3
BAFTA Awards: 1
BAFTA Nominations: 2
Golden Globes: 0
Golden Globe Nominations: 0

Rebel Without a Cause
98. Rebel Without a Cause (1955)
PG-13 | 111 min | Drama

 7.6  Rate 89 Metascore
A rebellious young man with a troubled past comes to a new town, finding friends and enemies.

Director: Nicholas Ray | Stars: James Dean, Natalie Wood, Sal Mineo, Jim Backus

Votes: 97,387

*****
Actors: 4.7 Stars
Direction: 4.7 Stars
Screenplay: 4.8 Stars

Oscars: 0
Oscar Nominations: 3
BAFTA Awards: 0
BAFTA Nominations: 2
Golden Globes: 0
Golden Globe Nominations: 0

North by Northwest
99. North by Northwest (1959)
Approved | 136 min | Action, Adventure, Mystery

 8.3  Rate 98 Metascore
A New York City advertising executive goes on the run after being mistaken for a government agent by a group of foreign spies, and falls for a woman whose loyalties he begins to doubt.

Director: Alfred Hitchcock | Stars: Cary Grant, Eva Marie Saint, James Mason, Jessie Royce Landis

Votes: 344,187 | Gross: $13.28M

Actors: 4.8 Stars
Direction: 4.9 Stars
Screenplay: 4.8 Stars

Oscars: 0
Oscar Nominations: 3
BAFTA Awards: 0
BAFTA Nominations: 0
Golden Globes: 0
Golden Globe Nominations: 0

Yankee Doodle Dandy
100. Yankee Doodle Dandy (1942)
Passed | 126 min | Biography, Drama, Family

 7.6  Rate 89 Metascore
The life of the renowned musical composer, playwright, actor, dancer, and singer George M. Cohan.

Director: Michael Curtiz | Stars: James Cagney, Joan Leslie, Walter Huston, Richard Whorf

Votes: 16,761 | Gross: $11.80M

Actors: 4.6 Stars
Direction: 4.7 Stars
Screenplay: 4.6 Stars

Oscars: 3
Oscar Nominations: 8
BAFTA Awards: 0
BAFTA Nominations: 0
Golden Globes: 0
Golden Globe Nominations: 0
"""